## Declaraciones

In [1]:
# import
import pandas as pd
import numpy as np

# Joanne

---

# Daniel

---

# Joel

In [63]:
years = ['2009','2010','2011','2012','2013','2014','2015','2016']

In [64]:
urgencies = pd.DataFrame(columns=['CLUES','TIPOURGENCIA','MOTATE','YEAR','tmp_trail'])

for year in years:
    path = f'Datos/Urgencias_{year}/'
    file = f'URGENCIAS_{year}.csv'
    
    try:
        # code to process the 2009 file (it doesn't cointain headers)
        # ENTRESIDENCIA, MP y TIPOURGENCIA se empieza a reportar a partir del año 2010
        # MUNRESIDENCIA se empieza a reportar a partir del año 2012
        if year in ['2009']:
            print(f'[LOG]: Processing... {path+file}')
            urgencies_tmp = pd.read_csv(path+file, sep=";", header=None, 
                    names=['ID','CLUES','FOLIO','FECHAALTA',
                    'EDAD','CVEEDAD','SEXO',#'ENTRESIDENCIA','MUNRESIDENCIA',
                    'DERHAB',#'TIPOURGENCIA',
                    'MOTATE','TIPOCAMA','ENVIADOA',#'MP',
                    'AFECPRIN','IRA','PLANEDA','SOBRESEDA',
                    'FECHAINGRESO','HORASESTANCIA','MES_ESTADISTICO',
                    'HORAINIATE','MININIATE','HORATERATE','MINTERATE'],
                    dtype={'ID': object,'CLUES': object,'FOLIO': object,'FECHAALTA': object,
                    'EDAD': object,'CVEEDAD': object,'SEXO': object,#'ENTRESIDENCIA': object, 'MUNRESIDENCIA': object,
                    'DERHAB': object,#'TIPOURGENCIA': np.float64,
                    'MOTATE': np.float64,'TIPOCAMA': object,'ENVIADOA': object,#'MP': object,
                    'AFECPRIN': object,'IRA': object,'PLANEDA': object,'SOBRESEDA': object,
                    'FECHAINGRESO': object,'HORASESTANCIA': object,'MES_ESTADISTICO': object,
                    'HORAINIATE': object,'MININIATE': object,'HORATERATE': object,'MINTERATE': object})
            print(f'[LOG]: {file} processed.')
        # code to process the 2010 & 2011 files (it doesn't cointain headers)
        # MUNRESIDENCIA se empieza a reportar a partir del año 2012
        elif year in ['2010','2011']:
            print(f'[LOG]: Processing... {path+file}')
            urgencies_tmp = pd.read_csv(path+file, sep=";", header=None, 
                    names=['ID','CLUES','FOLIO','FECHAALTA',
                    'EDAD','CVEEDAD','SEXO','ENTRESIDENCIA',
                    #'MUNRESIDENCIA',
                    'DERHAB','TIPOURGENCIA',
                    'MOTATE','TIPOCAMA','ENVIADOA','MP',
                    'AFECPRIN','IRA','PLANEDA','SOBRESEDA',
                    'FECHAINGRESO','HORASESTANCIA','MES_ESTADISTICO',
                    'HORAINIATE','MININIATE','HORATERATE','MINTERATE'],
                    dtype={'ID': object,'CLUES': object,'FOLIO': object,'FECHAALTA': object,
                    'EDAD': object,'CVEEDAD': object,'SEXO': object,'ENTRESIDENCIA': object,
                    #'MUNRESIDENCIA': object,
                    'DERHAB': object,'TIPOURGENCIA': np.float64,
                    'MOTATE': np.float64,'TIPOCAMA': object,'ENVIADOA': object,'MP': object,
                    'AFECPRIN': object,'IRA': object,'PLANEDA': object,'SOBRESEDA': object,
                    'FECHAINGRESO': object,'HORASESTANCIA': object,'MES_ESTADISTICO': object,
                    'HORAINIATE': object,'MININIATE': object,'HORATERATE': object,'MINTERATE': object})
            print(f'[LOG]: {file} processed.')
        # code to process the 2012, 2013 & 2014 file (it doesn't cointain headers)
        elif year in ['2012','2013','2014']:
            print(f'[LOG]: Processing... {path+file}')
            urgencies_tmp = pd.read_csv(path+file, sep=";", header=None, 
                    names=['ID','CLUES','FOLIO','FECHAALTA',
                    'EDAD','CVEEDAD','SEXO','ENTRESIDENCIA',
                    'MUNRESIDENCIA','DERHAB','TIPOURGENCIA',
                    'MOTATE','TIPOCAMA','ENVIADOA','MP',
                    'AFECPRIN','IRA','PLANEDA','SOBRESEDA',
                    'FECHAINGRESO','HORASESTANCIA','MES_ESTADISTICO',
                    'HORAINIATE','MININIATE','HORATERATE','MINTERATE'],
                    dtype={'ID': object,'CLUES': object,'FOLIO': object,'FECHAALTA': object,
                    'EDAD': object,'CVEEDAD': object,'SEXO': object,'ENTRESIDENCIA': object,
                    'MUNRESIDENCIA': object,'DERHAB': object,'TIPOURGENCIA': np.float64,
                    'MOTATE': np.float64,'TIPOCAMA': object,'ENVIADOA': object,'MP': object,
                    'AFECPRIN': object,'IRA': object,'PLANEDA': object,'SOBRESEDA': object,
                    'FECHAINGRESO': object,'HORASESTANCIA': object,'MES_ESTADISTICO': object,
                    'HORAINIATE': object,'MININIATE': object,'HORATERATE': object,'MINTERATE': object})
            print(f'[LOG]: {file} processed.')
        # code to process the 2015 & 2016 file 
        else:
            print(f'[LOG]: Processing... {path+file}')
            urgencies_tmp = pd.read_csv(path+file,
                    dtype={'ID': object,'CLUES': object,'FOLIO': object,'FECHAALTA': object,
                    'EDAD': object,'CVEEDAD': object,'SEXO': object,'ENTRESIDENCIA': object,
                    'MUNRESIDENCIA': object,'DERHAB': object,'TIPOURGENCIA': np.float64,
                    'MOTATE': np.float64,'TIPOCAMA': object,'ENVIADOA': object,'MP': object,
                    'AFECPRIN': object,'IRA': object,'PLANEDA': object,'SOBRESEDA': object,
                    'FECHAINGRESO': object,'HORASESTANCIA': object,'MES_ESTADISTICO': object,
                    'HORAINIATE': object,'MININIATE': object,'HORATERATE': object,'MINTERATE': object})
            print(f'[LOG]: {file} processed.')
        
        if year in ['2009']:
            urgencies_tmp['TIPOURGENCIA'] = -1
        
        urgencies_tmp = urgencies_tmp[['CLUES','TIPOURGENCIA','MOTATE']]
        urgencies_tmp['YEAR'] = year
        urgencies_tmp['tmp_trail'] = 1
        
        print(f'[LOG]: Selected: {file}')
        
        urgencies = pd.concat([urgencies, urgencies_tmp], ignore_index=True)
        
        
        print(f'[LOG]: Appended: {file}')
        
        
    except:
        print(f'[LOG]: Error: {file}')
    

[LOG]: Processing... Datos/Urgencias_2009/URGENCIAS_2009.csv
[LOG]: URGENCIAS_2009.csv processed.
[LOG]: Selected: URGENCIAS_2009.csv
[LOG]: Appended: URGENCIAS_2009.csv
[LOG]: Processing... Datos/Urgencias_2010/URGENCIAS_2010.csv
[LOG]: URGENCIAS_2010.csv processed.
[LOG]: Selected: URGENCIAS_2010.csv
[LOG]: Appended: URGENCIAS_2010.csv
[LOG]: Processing... Datos/Urgencias_2011/URGENCIAS_2011.csv
[LOG]: URGENCIAS_2011.csv processed.
[LOG]: Selected: URGENCIAS_2011.csv
[LOG]: Appended: URGENCIAS_2011.csv
[LOG]: Processing... Datos/Urgencias_2012/URGENCIAS_2012.csv
[LOG]: URGENCIAS_2012.csv processed.
[LOG]: Selected: URGENCIAS_2012.csv
[LOG]: Appended: URGENCIAS_2012.csv
[LOG]: Processing... Datos/Urgencias_2013/URGENCIAS_2013.csv
[LOG]: URGENCIAS_2013.csv processed.
[LOG]: Selected: URGENCIAS_2013.csv
[LOG]: Appended: URGENCIAS_2013.csv
[LOG]: Processing... Datos/Urgencias_2014/URGENCIAS_2014.csv
[LOG]: URGENCIAS_2014.csv processed.
[LOG]: Selected: URGENCIAS_2014.csv
[LOG]: Appended:

In [65]:
urgencies_by_year = urgencies.groupby(['YEAR'])["tmp_trail"].count().reset_index(name="count")

urgencies_by_year

,YEAR,count
0,2009,7344442
1,2010,7816941
2,2011,8579204
3,2012,9510442
4,2013,10041694
5,2014,10645625
6,2015,10918860
7,2016,10554908


In [66]:
urgencies.head()

,CLUES,TIPOURGENCIA,MOTATE,YEAR,tmp_trail
0,CCSSA000100,-1,6.0,2009,1
1,CCSSA000100,-1,6.0,2009,1
2,CCSSA000100,-1,6.0,2009,1
3,CCSSA000100,-1,6.0,2009,1
4,CCSSA000100,-1,6.0,2009,1


In [67]:
urgencies.count()

CLUES           75412116
TIPOURGENCIA    75412116
MOTATE          72899234
YEAR            75412116
tmp_trail       75412116
dtype: int64

In [68]:
urgencies_cdmx = urgencies[(urgencies['CLUES'].str.contains("DF"))]

urgencies_cdmx.count()

CLUES           7017663
TIPOURGENCIA    7017663
MOTATE          6934208
YEAR            7017663
tmp_trail       7017663
dtype: int64

In [254]:
urgencies_cdmx_grouped = urgencies_cdmx.groupby(['CLUES','TIPOURGENCIA','MOTATE','YEAR'])["tmp_trail"].count().reset_index(name="count")

urgencies_cdmx_grouped.head()

,CLUES,TIPOURGENCIA,MOTATE,YEAR,count
0,DFSSA000053,-1.0,1.0,2009,1278
1,DFSSA000053,-1.0,2.0,2009,8524
2,DFSSA000053,-1.0,3.0,2009,19
3,DFSSA000053,-1.0,5.0,2009,4
4,DFSSA000053,-1.0,6.0,2009,24029


In [255]:
urgencies_cdmx_grouped.count()

CLUES           2184
TIPOURGENCIA    2184
MOTATE          2184
YEAR            2184
count           2184
dtype: int64

In [256]:
output_file = 'data_output/urgencies_by_date.csv'

urgencies_cdmx_grouped.to_csv(output_file, encoding='utf-8', index=False)

---

In [257]:
### Upload the file  (to not process all the info twice)

#output_file = 'data_output/urgencies_by_date.csv'

#urgencies_cdmx_grouped = pd.read_csv(output_file)

In [258]:
urgencies_cdmx_grouped.head()

,CLUES,TIPOURGENCIA,MOTATE,YEAR,count
0,DFSSA000053,-1.0,1.0,2009,1278
1,DFSSA000053,-1.0,2.0,2009,8524
2,DFSSA000053,-1.0,3.0,2009,19
3,DFSSA000053,-1.0,5.0,2009,4
4,DFSSA000053,-1.0,6.0,2009,24029


In [259]:

CatTipoUrgencia = {1:"URGENCIA CALIFICADA", 
                    2:"URGENCIA NO CALIFICADA", 
                    3:"APOYO A SERVICIOS DE MEDICINA GENERAL"}

CatMotAtencionUrg = {1:"ACCIDENTES, ENVENENAMIENTO Y VIOLENCIAS", 
                    2:"MEDICA", 
                    3:"GINECO-OBSTETRICA", 
                    4:"PEDIATRICA", 
                    9:"NO ESPECIFICADO"}

CatMotAtencionUrg_2008 = {1:"(AEV) ACCIDENTES, ENVENENAMIENTO Y VIOLENCIAS",
                    2:"(UC) URGENCIA CALIFICADA",
                    3:"(UNC) URGENCIA NO CALIFICADA",
                    4:"TRABAJO DE PARTO",
                    9:"NO ESPECIFICADO"}

CatMotAtencionUrg_2009 = {1:"ACCIDENTES, ENVENENAMIENTO Y VIOLENCIAS",
                    2:"URGENCIA CALIFICADA MEDICA",
                    3:"URGENCIA CALIFICADA QUIRURGICA",
                    4:"URGENCIA CALIFICADA GINECO-OBSTETRICA",
                    5:"URGENCIA CALIFICADA PEDIATRICA",
                    6:"URGENCIA NO CALIFICADA",
                    7:"APOYO A SERVICIOS DE MEDICINA GENERAL",
                    9:"NO ESPECIFICADO"}



In [260]:
urgencies_cdmx_grouped['TIPOURGENCIA_V'] = ''
urgencies_cdmx_grouped['MOTATE_V'] = ''

for key, value in CatTipoUrgencia.items():
    print(f'{key} - {value}')
    urgencies_cdmx_grouped['TIPOURGENCIA_V'] = np.where(urgencies_cdmx_grouped['TIPOURGENCIA']==key, 
                                                        value, urgencies_cdmx_grouped['TIPOURGENCIA_V'])
    
print('------')    

for key, value in CatMotAtencionUrg.items():
    print(f'{key} - {value}')
    urgencies_cdmx_grouped['MOTATE_V'] = np.where( (urgencies_cdmx_grouped['MOTATE']==key), 
                                                  value, urgencies_cdmx_grouped['MOTATE_V'])

print('------')    

for key, value in CatMotAtencionUrg_2009.items():
    print(f'{key} - {value}')
    urgencies_cdmx_grouped['MOTATE_V'] = np.where((urgencies_cdmx_grouped['MOTATE']==key) & 
                                                  (urgencies_cdmx_grouped['YEAR']=='2009'), 
                                                  value, urgencies_cdmx_grouped['MOTATE_V'])

print('------ Fix 2009 - 1 - URGENCIA CALIFICADA')    


urgencies_cdmx_grouped['TIPOURGENCIA'] = np.where((urgencies_cdmx_grouped['MOTATE'].isin([2,3,4,5])) &
                                                    (urgencies_cdmx_grouped['YEAR']=='2009'), 
                                                    1, urgencies_cdmx_grouped['TIPOURGENCIA'])

print('------ Fix 2009 - 2 - URGENCIA NO CALIFICADA')    


urgencies_cdmx_grouped['TIPOURGENCIA'] = np.where((urgencies_cdmx_grouped['MOTATE'] == 6) &
                                                    (urgencies_cdmx_grouped['YEAR']=='2009'), 
                                                    2, urgencies_cdmx_grouped['TIPOURGENCIA'])


print('------ Fix 2009 - 3 - APOYO A SERVICIOS DE MEDICINA GENERAL')    


urgencies_cdmx_grouped['TIPOURGENCIA'] = np.where((urgencies_cdmx_grouped['MOTATE'] == 7) &
                                                    (urgencies_cdmx_grouped['YEAR']=='2009'), 
                                                    3, urgencies_cdmx_grouped['TIPOURGENCIA'])

    
print('------')  
    
for key, value in CatTipoUrgencia.items():
    print(f'{key} - {value}')
    urgencies_cdmx_grouped['TIPOURGENCIA_V'] = np.where( (urgencies_cdmx_grouped['TIPOURGENCIA']==key) &
                                                        (urgencies_cdmx_grouped['YEAR']=='2009'), 
                                                        value, urgencies_cdmx_grouped['TIPOURGENCIA_V'])

print('------ Fix 2009 MOTATE Fix') 
urgencies_cdmx_grouped['MOTATE_V'] = np.where((urgencies_cdmx_grouped['MOTATE_V']=="URGENCIA CALIFICADA MEDICA") & 
                                                  (urgencies_cdmx_grouped['YEAR']=='2009'), 
                                                  "MEDICA", urgencies_cdmx_grouped['MOTATE_V'])

urgencies_cdmx_grouped['MOTATE_V'] = np.where((urgencies_cdmx_grouped['MOTATE_V']=="URGENCIA CALIFICADA QUIRURGICA") & 
                                                  (urgencies_cdmx_grouped['YEAR']=='2009'), 
                                                  "QUIRURGICA", urgencies_cdmx_grouped['MOTATE_V'])

urgencies_cdmx_grouped['MOTATE_V'] = np.where((urgencies_cdmx_grouped['MOTATE_V']=="URGENCIA CALIFICADA GINECO-OBSTETRICA") & 
                                                  (urgencies_cdmx_grouped['YEAR']=='2009'), 
                                                  "GINECO-OBSTETRICA", urgencies_cdmx_grouped['MOTATE_V'])

urgencies_cdmx_grouped['MOTATE_V'] = np.where((urgencies_cdmx_grouped['MOTATE_V']=="URGENCIA CALIFICADA PEDIATRICA") & 
                                                  (urgencies_cdmx_grouped['YEAR']=='2009'), 
                                                  "PEDIATRICA", urgencies_cdmx_grouped['MOTATE_V'])
    


1 - URGENCIA CALIFICADA
2 - URGENCIA NO CALIFICADA
3 - APOYO A SERVICIOS DE MEDICINA GENERAL
------
1 - ACCIDENTES, ENVENENAMIENTO Y VIOLENCIAS
2 - MEDICA
3 - GINECO-OBSTETRICA
4 - PEDIATRICA
9 - NO ESPECIFICADO
------
1 - ACCIDENTES, ENVENENAMIENTO Y VIOLENCIAS
2 - URGENCIA CALIFICADA MEDICA
3 - URGENCIA CALIFICADA QUIRURGICA
4 - URGENCIA CALIFICADA GINECO-OBSTETRICA
5 - URGENCIA CALIFICADA PEDIATRICA
6 - URGENCIA NO CALIFICADA
7 - APOYO A SERVICIOS DE MEDICINA GENERAL
9 - NO ESPECIFICADO
------ Fix 2009 - 1 - URGENCIA CALIFICADA
------ Fix 2009 - 2 - URGENCIA NO CALIFICADA
------ Fix 2009 - 3 - APOYO A SERVICIOS DE MEDICINA GENERAL
------
1 - URGENCIA CALIFICADA
2 - URGENCIA NO CALIFICADA
3 - APOYO A SERVICIOS DE MEDICINA GENERAL
------ Fix 2009 MOTATE Fix


---

In [261]:
urgencies_cdmx_grouped.head()

,CLUES,TIPOURGENCIA,MOTATE,YEAR,count,TIPOURGENCIA_V,MOTATE_V
0,DFSSA000053,-1.0,1.0,2009,1278,,"ACCIDENTES, ENVENENAMIENTO Y VIOLENCIAS"
1,DFSSA000053,1.0,2.0,2009,8524,URGENCIA CALIFICADA,MEDICA
2,DFSSA000053,1.0,3.0,2009,19,URGENCIA CALIFICADA,QUIRURGICA
3,DFSSA000053,1.0,5.0,2009,4,URGENCIA CALIFICADA,PEDIATRICA
4,DFSSA000053,2.0,6.0,2009,24029,URGENCIA NO CALIFICADA,URGENCIA NO CALIFICADA


In [262]:
print(urgencies_cdmx_grouped.TIPOURGENCIA.unique())
print(urgencies_cdmx_grouped.TIPOURGENCIA_V.unique())

[-1.  1.  2.  3.]
['' 'URGENCIA CALIFICADA' 'URGENCIA NO CALIFICADA'
 'APOYO A SERVICIOS DE MEDICINA GENERAL']


In [263]:
print(urgencies_cdmx_grouped.MOTATE.unique())
print(urgencies_cdmx_grouped.MOTATE_V.unique())

[1. 2. 3. 5. 6. 9. 4. 7.]
['ACCIDENTES, ENVENENAMIENTO Y VIOLENCIAS' 'MEDICA' 'QUIRURGICA'
 'PEDIATRICA' 'URGENCIA NO CALIFICADA' 'NO ESPECIFICADO'
 'GINECO-OBSTETRICA' 'APOYO A SERVICIOS DE MEDICINA GENERAL']


---

In [264]:
empty = urgencies_cdmx_grouped[(urgencies_cdmx_grouped['MOTATE_V'] == '' )]

empty.count()

CLUES             0
TIPOURGENCIA      0
MOTATE            0
YEAR              0
count             0
TIPOURGENCIA_V    0
MOTATE_V          0
dtype: int64

In [265]:
empty = urgencies_cdmx_grouped[(urgencies_cdmx_grouped['TIPOURGENCIA_V'] == '' )]

empty.count()

CLUES             48
TIPOURGENCIA      48
MOTATE            48
YEAR              48
count             48
TIPOURGENCIA_V    48
MOTATE_V          48
dtype: int64

In [266]:
output_file = 'data_output/urgencies_by_date_cat.csv'

urgencies_cdmx_grouped.to_csv(output_file, encoding='utf-8', index=False)